# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [27]:
!pwd

/home/gongai/projects/wgong/py4kids/lesson-18-ai/crewai/papagame


In [1]:
from IPython.display import Markdown, display
import os
from time import time
import json
from pprint import pprint

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, MDXSearchTool,
    DirectoryReadTool, FileReadTool, BaseTool
)

from langchain_openai import ChatOpenAI

from pydantic import BaseModel

from api_key_store import ApiKeyStore


# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from helper import *

## Setup

### Inputs

In [3]:
# inputs json doc
# 
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """
        Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
        managing remote and in-office teams, and expert in multiple programming languages and frameworks. 
        He holds an MBA and a strong background in AI and data science. Noah has successfully led
        major tech initiatives and startups, proving his ability to drive innovation and growth in the tech industry. 
        Ideal for leadership roles that require a strategic and innovative approach.
    """
}

### Outputs

In [4]:
folder_name = "job-app"
create_folder(folder_name)

In [5]:
file_in_draft_resume = Path(folder_name) / 'fake_resume.md'
file_out_interview_material = Path(folder_name) / "interview_materials.md"
file_out_tailored_resume = Path(folder_name) / "tailored_resume.md"

In [6]:
file_in_draft_resume, str(file_in_draft_resume), file_in_draft_resume.name

(PosixPath('job-app/fake_resume.md'),
 'job-app/fake_resume.md',
 'fake_resume.md')

### LLM and keys

In [7]:
s = ApiKeyStore()

os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI/Yiwen")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

model_name = "gpt-3.5-turbo"
os.environ["OPENAI_MODEL_NAME"] = model_name   # gpt-4 was default, throws rate-limit error 

## Define Tools

In [8]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=file_in_draft_resume)
semantic_search_resume = MDXSearchTool(mdx=str(file_in_draft_resume))

Inserting batches in chromadb: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.62it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [9]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Create Agents

In [10]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on  job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory="""
        "As a Job Researcher, your prowess in  navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary  qualifications and skills sought "
        "by employers, forming the foundation for  effective application tailoring."
    """.replace('"',''),
)

In [11]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants  to help them stand out in the job market",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "Equipped with analytical prowess, you dissect  and synthesize information "
        "from diverse sources to craft comprehensive  personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    """.replace('"',''),
)

In [12]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a  resume stand out in the job market.",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "With a strategic mind and an eye for detail, you  excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they  resonate perfectly with the job's requirements."
    """.replace('"',''),
)

In [13]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points  based on the resume and job requirements",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "Your role is crucial in anticipating the dynamics of  interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success,  ensuring they can confidently address all aspects of the "
        "job they are applying for."
    """.replace('"',''),
)

## Create Tasks

In [14]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description="""
        "Analyze the job posting URL provided ({job_posting_url})  to extract required key skills, experiences, and qualifications. "
        "Use the tools to gather content and identify  and categorize the requirements."
    """.replace('"',''),
    expected_output="A structured list of job requirements, including necessary skills, qualifications, and experiences.",
    agent=researcher,
    async_execution=True
)

In [15]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description="""
        "Compile a detailed personal and professional profile  using the GitHub ({github_url}) URLs, "
        "and personal write-up ({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    """.replace('"',''),
    expected_output="""
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and communication style."
    """.replace('"',''),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [16]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
# depends on research_task, profile_task
resume_strategy_task = Task(
    description="""
        "Using the profile and job requirements obtained from previous tasks,"
        " tailor the resume to highlight the most relevant areas. "
        "Employ tools to adjust and enhance the resume content. "
        "Make sure this is the best resume even but don't make up any information. "
        "Update every section,  inlcuding the initial summary, work experience, skills, and education. "
        "All to better reflrect the candidates  abilities and how it matches the job posting."
    """.replace('"',''),
    expected_output="""
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    """.replace('"',''),
    output_file=str(file_out_tailored_resume),
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [17]:
# Task for Interview Preparer Agent: Develop Interview Materials
# depends on research_task, profile_task, resume_strategy_task
interview_preparation_task = Task(
    description="""
        "Create a set of potential interview questions and talking  points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion points. "
        "Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume  and how it matches the job posting."
    """.replace('"',''),
    expected_output="""
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    """.replace('"',''),
    output_file=str(file_out_interview_material),
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Create the Crew

In [18]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Run the Crew Workflow

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
### this execution will take a few minutes to run
ts_start = time()
result = job_application_crew.kickoff(inputs=job_application_inputs)
ts_stop = time()

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: 
        Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund)  to extract required key skills, experiences, and qualifications. 
        Use the tools to gather content and identify  and categorize the requirements.
    
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: 
        Compile a detailed personal and professional profile  using the GitHub (https://github.com/joaomdmoura) URLs, 
        and personal write-up (
        Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
        managing remote and in-office teams, and expert in multiple programming languages and frameworks. 
        He holds an MBA and a strong background in AI and data science. Noah has successfully led
    

In [24]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")   # Elapsed time: 53.83 sec

# 26.14 sec on 2024-06-29



Elapsed time: 26.14 sec


- Dislplay the generated `tailored_resume.md` file.

In [25]:

display(Markdown(file_out_tailored_resume))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

- Dislplay the generated `interview_materials.md` file.

In [26]:
display(Markdown(file_out_interview_material))

# Interview Questions and Talking Points for Noah Williams

## Experience and Skills:
1. Can you elaborate on your experience as a Full Stack Developer and how it has prepared you for leadership roles in software engineering?
2. How have you utilized your expertise in managing remote and in-office engineering teams to drive innovation and growth in previous positions?
3. In what ways have you leveraged your proficiency in multiple programming languages and frameworks to enhance project outcomes and team collaboration?
4. Could you share a specific example of a project where you successfully integrated AI technologies and data science to achieve business objectives?
5. How do you approach strategic planning and execution in your role as a Software Engineering Leader?

## Project Experiences:
1. What strategies did you implement at EliteDevs to enhance inter-departmental coordination and trust within the engineering teams?
2. How did you lead the high-performance engineering team at PrintPack to increase company revenue by 500%?
3. Can you discuss the impact of integrating data analytics into business decision-making processes at PrintPack?
4. What challenges did you face when transforming the engineering division at DataKernel into a key revenue pillar, and how did you overcome them?
5. How have you led skill development and strategic project goal achievement at DataKernel?

## Contributions and Achievements:
1. How were you instrumental in enhancing inter-departmental coordination and trust in your previous roles?
2. What pivotal contributions did you make to increase company revenue and develop predictive modeling tools?
3. What cutting-edge AI technologies and scalable vector databases did you integrate into your projects, and what were the results?
4. How do you balance productivity with innovation and prioritize long-term company goals in your leadership approach?

## Interests and Communication Style:
1. How do your interests in AI and data science influence your approach to leading engineering teams?
2. What strategies do you employ in remote team management to ensure effective collaboration and communication?
3. How do you incorporate strategic planning and innovation into your daily work as a Software Engineering Leader?
4. Can you provide examples of how you use Ruby on Rails and React to build new products and foster innovation?

These questions and talking points will help Noah Williams showcase his skills, experiences, and achievements during the interview process, highlighting his suitability for leadership roles in the tech industry.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)